In [ ]:
import json
from dataclasses import asdict
from typing import List
import re
from CTnlp.patient import load_patients_from_xml
from CTnlp.patient import Patient

In [ ]:

def convert_patients_to_jsonl(patients: List[Patient], outfile):
    with open(outfile, 'w') as fp:
        for patient in patients:
            fp.write(json.dumps(asdict(patient)))
            fp.write('\n')

In [ ]:
patients = []
patients.extend(load_patients_from_xml("../data/external/topics2014.xml", input_type="CSIRO"))
patients.extend(load_patients_from_xml("../data/external/topics2021.xml"))
patients.extend(load_patients_from_xml("../data/external/topics2022.xml"))

In [ ]:
patients[0]

In [ ]:
def extract_past_medical_history(patient):
    description = patient.description.lower()
    match = re.search(r"[!\.][^!\.]*medical history.*?\.", description, re.IGNORECASE)
    if not match:
        match = re.search(r"[!\.][^!\.]*has (no )?(a )?(positive )?history.*?\.", description, re.IGNORECASE)
    if not match:
        match = re.search(r"[!\.][^!\.]*past medical history:?\n([\d|-]?[^\n]*\n)*", description, re.IGNORECASE)
    return match

In [ ]:
def extract_family_history(patient):
    description = patient.description.lower()
    match = re.search(r"\.[^\.]*family history.*?\.", description, re.IGNORECASE)
    return match

In [ ]:
def extract_sections(patient):
    _description = patient.description
    rest = patient.description
    pmh_text = ''
    fh_text = ''

    pmh = extract_past_medical_history(patient)
    fh = extract_family_history(patient)

    if pmh and fh:
        if pmh.start() > fh.start():
            _first = fh
            _second = pmh
        else:
            _first = pmh
            _second = fh

        rest = _description[:_second.start()+2] + _description[_second.end()+1:]
        rest = rest[:_first.start()+2] + rest[_first.end()+1:]
        pmh_text = _description[pmh.start()+1:pmh.end()].strip()
        fh_text = _description[fh.start()+1:fh.end()].strip()

    if pmh and not fh:
        rest = _description[:pmh.start()+2] + _description[pmh.end()+1:]
        pmh_text = _description[pmh.start()+1:pmh.end()].strip()

    if fh and not pmh:
        rest = _description[:fh.start()+2] + _description[fh.end()+1:]
        fh_text = _description[fh.start()+1:fh.end()].strip()

    print(f"{pmh=}\t{fh=}")

    return rest, pmh_text, fh_text

In [ ]:
out_dict = []
for patient in patients:
    rest, pmh, fh = extract_sections(patient)
    patient.pmh = pmh
    patient.fh = fh
    patient.rest = rest

    out_dict.append({
        'patient_id': patient.patient_id,
        'description':rest,
        'pmh':pmh,'fh':fh})

In [ ]:
OUTFILE = '../data/external/sections.jsonl'
# convert_patients_to_jsonl(patients=patients, outfile=OUTFILE)

In [ ]:
with open(OUTFILE, 'w') as fp:
    for item in out_dict:
        fp.write(json.dumps(item))
        fp.write('\n')

In [ ]:
rest

In [ ]:
pmh

# Medspacy testing

In [ ]:
import medspacy
from medspacy.context import ConTextRule, ConTextComponent
# from medspacy.visualization import visualize_dep, visualize_ent
# from scispacy.linking import EntityLinker


In [ ]:
nlp = medspacy.load("en_ner_bc5cdr_md", enable=["sentencizer", "context"])

In [ ]:
# nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

In [ ]:
# nlp = medspacy.load(enable=["sentencizer", "context"])

In [ ]:
context = ConTextComponent(nlp, rules="default")

In [ ]:
context.rules

In [ ]:
doc = nlp("She has no allergies to any food or drugs. There is abscess in the abdomen. There is a collection of fluid in the jejunum. hematomas are seen around the right lower quadrant. There is no cancer.")

In [ ]:
context(doc)

In [ ]:
doc._.context_graph

In [ ]:
doc._.context_graph.targets

In [ ]:
doc._.context_graph.modifiers

In [ ]:
from medspacy.visualization import visualize_dep, visualize_ent

In [ ]:
visualize_ent(doc)

In [ ]:
visualize_dep(doc)

In [ ]:
nlp = medspacy.load("en_info_3700_i2b2_2012", enable=['sentencizer', 'tagger', 'parser',
                                                      'ner', 'target_matcher', 'context',
                                                     'sectionizer'])

In [ ]:
expanded = []
for patient in patients[120:130]:
    doc = nlp(patient.description)
    # visualize_ent(doc)
    for target, modifier in doc._.context_graph.edges:
        print("[{0}] is modified by [{1}]".format(target, modifier))
        if modifier.category == 'NEGATED_EXISTENCE':
            expanded.append(f"no_{'_'.join(str(target).lower().split())}")
    # print(doc._.context_graph)

In [ ]:
doc._.context_graph.targets

In [ ]:
doc._.context_graph.targets

In [ ]:
doc.ents

In [ ]:
visualize_ent(doc)

In [ ]:
expanded

In [ ]:
from CTnlp.parsers import parse_clinical_trials_from_folder
cts = parse_clinical_trials_from_folder("/Users/wojciechkusa/projects/shared-tasks/clinical-trials/data/external/")
len(cts)

In [ ]:
for ct in cts:
    doc = nlp(ct.criteria)
    visualize_ent(doc)

In [ ]:
def get_entities(text):
    doc = nlp(text)

    entities = []
    negated_entities = []
    pmh_entities = []
    fh_entities = []
    for ent in doc.ents:
        if any([ent._.is_negated, ent._.is_uncertain, ent._.is_historical, ent._.is_family, ent._.is_hypothetical, ]):
            # print("'{0}' modified by {1} in: '{2}'".format(ent, ent._.modifiers, ent.sent))
            # print()
            if ent._.is_negated:
                # negated_entities.append(f"no_{'_'.join(str(ent).lower().split())}")
                negated_entities.append(str(ent))
            elif ent._.is_historical:
                pmh_entities.append(str(ent))
            elif ent._.is_family:
                fh_entities.append(str(ent))
            else:
                entities.append(str(ent))
        else:
            entities.append(str(ent))

    return {"entities": entities, "negated_entities": negated_entities, "pmh_entities": pmh_entities, "fh_entities": fh_entities,}

In [ ]:
output_entities = []
for patient in patients:
    entities_dict = get_entities(patient.description)
    entities_dict["id"] = patient.patient_id
    output_entities.append(entities_dict)

In [ ]:
output_entities

In [ ]:
OUTFILE = '../data/external/entities.jsonl'

In [ ]:
with open(OUTFILE, 'w') as fp:
    for item in output_entities:
        fp.write(json.dumps(item))
        fp.write('\n')

In [ ]:
cts[2].conditions

In [ ]:
patient.description

In [ ]:
doc = nlp(patient.description)

In [ ]:
for ent in doc.ents:
    if any([ent._.is_negated, ent._.is_uncertain, ent._.is_historical, ent._.is_family, ent._.is_hypothetical, ]):
        print(ent, [ent._.is_negated, ent._.is_uncertain, ent._.is_historical, ent._.is_family, ent._.is_hypothetical, ])

In [ ]:
doc.ents